In [1469]:
# Import dependencies
import pandas as pd
import numpy as np
import json
import re

from os import listdir
from os.path import isfile, join
from collections import Counter
from datetime import datetime
from matplotlib import pyplot as plt

import plotly.express as px

## Data Retrieval

In [1470]:
# Define directory path
mypath = "./data_dir"

# Create list of json file names in directory
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [1471]:
# Initialize empty list for all debate data
all_debates_list = []

# Loop through and read in all json files
for file in onlyfiles:
    
    # Open next JSON file
    with open(f"{mypath}/{file}") as json_data:

        # Load data from current JSON file
        data = json.load(json_data)

        # Fix dates for 1992 debate parts 1 and 2
        if (data['date'][0] == 'Part'):
            if data['date'][1] == '1':
                data['date'] = ['October', '11', '1992']
            elif data['date'][1] == '2':
                data['date'] = ['October', '15', '1992']

        # Append to full debate data list
        all_debates_list.append(data)

In [1472]:
# Get date in datetime format from a particular debate JSON
def get_date(debate):
    
    # Create date string from provided date array
    date_string = '-'.join(debate['date'])

    # Create date object from date string
    date_object = datetime.strptime(date_string, '%B-%d-%Y')

    # Return date of given debate in datetime
    return date_object

In [1473]:
# Get list of unique actors from a particular debate JSON
def get_unique_actors(debate):
    # Empty list to store all actors names (including duplicates)
    actor_list = []

    # Create list of prefixes to remove from names
    prefixes = ['Mr.', 'Ms.', 'Senator', 'Governor', 'Admiral']

    # Create dictionary of actor name corrections to be made
    typo_corrections = {
        # 'The President':'Reagan',
        '^Obam$':'Obama',
        'Barbara Walters':'Walters',
        'Bill Shadel': 'Shadel',
        'Edwin Newman': 'Newman',
        'Frank Mcgee': 'McGee',
        'Hal Bruno': 'Bruno',
        'Harry Ellis': 'Ellis',
        'Jim Lehrer': 'Lehrer',
        'Quincy Howe': 'Howe',
        'Sander Vanocur': 'Vanocur',
        'President Bush': 'Bush',
        '^Frederic$': 'Frederick'
    }

    # Loop through each speaking turn in the debate transcription content
    for turn in debate['content']:

        # Store actor name for each speaking turn
        actor = turn['actor']

        # Loop through prefixes in list
        for prefix in prefixes:

            # Check if the actor's name contains current prefix
            if prefix in actor:

                # Remove prefix and strip whitespace
                actor = actor.replace(prefix, '').strip()

        # Append actor to list of non-unique names
        actor_list.append(actor)

    # Loop through enumerated list of actor names
    for index, data in enumerate(actor_list):

        # Loop through items in typo corrections dict
        for k, v in typo_corrections.items():

            # Check if typo is present in current actor name
            if k in data:

                # Correct typo
                actor_list[index]=data.replace(k, typo_corrections[k])

    # Create unique list of cleaned actor names
    cleaned_actors = list(set(actor_list))

    # Return list of unique actor names for given debate
    return cleaned_actors

In [1474]:
# Get non-unique words given actor name and debate JSON data
def get_actor_dialogue(debate, actor):

    # Initialize actor's dialogue to empty string
    filtered_dialogue = ''

    # Initialize speaking turn counter to 0
    speaking_turn_count = 0

    # Loop through speaking turns in debate JSON data
    for turn in debate['content']:
        
        # Check if speaking turn is made by specified actor
        if actor in turn['actor'].split():

            # Increment actor's speaking turn count
            speaking_turn_count += 1

            # Concatenate to running dialogue string for selected actor
            filtered_dialogue += f"{turn['dialogue']} "
    # Create nonunique word list from dialogue with punctuation and whitespace removed
    turn_dialogue_list = re.sub(r'[^\w\s]', '', filtered_dialogue).split()
    
    # Make lowercase list of all words
    dialogue_list_lower = [word.lower() for word in turn_dialogue_list]

    
    # Return actor's speaking turn count and nonunique word list
    return speaking_turn_count, dialogue_list_lower

In [1475]:
### Collect data for dataframe

# Initialize row data for dataframe to empty list
row_data_list = []

# Loop through all debate data
for debate in all_debates_list:

    # Store date of currently selected debate
    debate_date = get_date(debate)

    # Loop through unique actor list for each debate
    for actor in get_unique_actors(debate):
        
        # Store speaking turn count and nonunique word list for currently selected debate/actor
        speaking_turn_count, actor_dialogue_list = get_actor_dialogue(debate, actor)

        # Store nonunique word count for selected debate/actor
        total_word_count = len(actor_dialogue_list)
    
        # Store unique word count for selected debate/actor
        unique_word_count = len(set(actor_dialogue_list))

        # Create row of relevant data for selected debate/actor
        row_data = [actor, debate_date, speaking_turn_count, total_word_count, unique_word_count]

        # Append row data to list
        row_data_list.append(row_data)

In [1476]:
# Create dataframe from collected row data
debate_dialogue_df = pd.DataFrame(row_data_list, columns=['actor', 'date', 'speaking_turn_count','total_word_count', 'unique_word_count'])

# Preview dataframe
debate_dialogue_df.head()

,actor,date,speaking_turn_count,total_word_count,unique_word_count
49,Mondale,1984-10-21,25,5128,1198
211,Mondale,1984-10-07,28,5080,1098


In [1477]:
# Create new column with calculated average number of words per turn
debate_dialogue_df['avg_words_per_turn'] = debate_dialogue_df['total_word_count'] / debate_dialogue_df['speaking_turn_count']

# Create new column with calculated average number of words per turn
debate_dialogue_df['avg_unique_words_per_turn'] = debate_dialogue_df['unique_word_count'] / debate_dialogue_df['speaking_turn_count']

# Fix Reagan's name in 1984
debate_dialogue_df['actor'].replace({"The President": "Reagan"})

# Preview dataframe
debate_dialogue_df = debate_dialogue_df.sort_values(by='date')

debate_dialogue_df.head()

,actor,date,speaking_turn_count,total_word_count,unique_word_count,avg_words_per_turn,avg_unique_words_per_turn
9,Nixon,1960-09-26,10,4111,820,411.100000,82.000000
5,Kennedy,1960-09-26,17,4693,953,276.058824,56.058824
6,Voice,1960-09-26,1,14,14,14.000000,14.000000
7,Fleming,1960-09-26,2,369,148,184.500000,74.000000
8,Warren,1960-09-26,4,268,166,67.000000,41.500000


In [1478]:
# Export dataframe as CSV
debate_dialogue_df.to_csv('./debate_data.csv', index=False)

In [1479]:
# Create function to count number of times each word was used
def actor_word_count(debate, actor):
    remove_words = ['the', 'to', 'of', 'in', 'and', 'that', 'a', 'is', 'for', 'it']
    turn_count, dialogue_list = get_actor_dialogue(debate, actor)

    dialogue_list_cleaned = [word for word in dialogue_list if word not in remove_words]

    c = Counter(dialogue_list_cleaned)

    return c

In [1480]:
debates_wiki_df = pd.read_html("http://en.wikipedia.org/wiki/United_States_presidential_debates")

candidates_table = debates_wiki_df[2]
viewership_table = debates_wiki_df[3]

## Data Cleaning

In [1481]:
vp_candidates = pd.DataFrame()
pres_candidates = pd.DataFrame()

candidates_table = candidates_table.rename(columns={"Election":"year",
                                "Presidential debates":"pres_debate_count",
                                "Presidential debates.1":"pres_candidate",
                                "Vice presidential debates":"vp_debate_count",
                                "Vice presidential debates.1":"vp_candidate"})

candidates_cleaned_df = candidates_table.drop(index=2)

pres_candidates[['year', 'debate_count', 'candidate']] = candidates_table[['year', 'pres_debate_count','pres_candidate']]
pres_candidates['type'] = 'P'
vp_candidates[['year', 'debate_count', 'candidate']] = candidates_table[['year', 'vp_debate_count','vp_candidate']]
vp_candidates['type'] = 'VP'

In [1482]:
candidates_df = pd.concat([pres_candidates, vp_candidates], ignore_index=True).sort_values(by='year').reset_index(drop=True)

candidates_df.drop(index=[56,57], inplace=True)

In [1483]:
candidates_df["debate_count"][candidates_df["year"] == '2020'] = '2'

candidates_df["debate_count"][candidates_df["debate_count"].str.contains('ebate', na=False)] = 0

candidates_df.head()

,year,debate_count,candidate,type
0,1960,4,Vice President Richard Nixon (R),P
1,1960,4,Senator John F. Kennedy (D),P
2,1960,0,No debates until 1976,VP
3,1960,0,No debates until 1976,VP
4,1976,3,President Gerald Ford (R),P


In [1484]:
candidates_df[['debate_count', 'year']] = candidates_df[['debate_count', 'year']].astype(int)

In [1485]:
# Remove rows with string in candidate column
candidates_df = candidates_df[candidates_df["candidate"].str.contains('ebate')==False]

In [1486]:
candidates_df[["candidate", "party"]] = candidates_df["candidate"].str.split("(", expand=True)
candidates_df["party"] = candidates_df["party"].str[0]
candidates_df['last_name'] = candidates_df['candidate'].str.split().str[-1]
candidates_df = candidates_df.reset_index(drop=True)


In [1487]:
won_election = [False, True, False, True, False, True, False, True, False, False,
                 True, True, False, False, True, True, False, False, True, False,
                 True, False, False, True, False, False, True, True, False, True,
                 False, False, False, True, True, True, False, True, False, False,
                 False, True, True, False, True, True, False, True, False, False,
                 True]

candidates_df.insert(6, 'won_election', won_election)

candidates_df.head()

,year,debate_count,candidate,type,party,last_name,won_election
0,1960,4,Vice President Richard Nixon,P,R,Nixon,False
1,1960,4,Senator John F. Kennedy,P,D,Kennedy,True
2,1976,3,President Gerald Ford,P,R,Ford,False
3,1976,3,Former Governor Jimmy Carter,P,D,Carter,True
4,1976,1,Senator Bob Dole,VP,R,Dole,False


In [1488]:
# Create list of candidate titles
titles = ['Former Vice President', 'Vice President', 'President',  'Former Senator', 'Senator', 'Former Governor',
          'Governor', 'Congressman', 'Congresswoman', 'Businessman', 'Ret. Vice Admiral', 'Former HUD Secretary',
          'Former Secretary of State']

# Define function to return the title string contained in candidate name
def title_split(a):
    for title in titles:
        if title in a:
            return title

# Create new title column
candidates_df['title'] = candidates_df['candidate'].apply(lambda x: title_split(x))

# Fix existing candidate column name to remove title
candidates_df['candidate'] = candidates_df['candidate'].apply(lambda x: x.split(title_split(x))[1]).str.strip()

In [1490]:
debate_dialogue_df['year'] = pd.DatetimeIndex(debate_dialogue_df['date']).year

# debate_dialogue_df = debate_dialogue_df.merge(candidates_df[['last_name', 'won_election']], left_on='actor', right_on='last_name')

debate_dialogue_df[debate_dialogue_df['actor']=='Mondale']

,actor,date,speaking_turn_count,total_word_count,unique_word_count,avg_words_per_turn,avg_unique_words_per_turn,year
211,Mondale,1984-10-07,28,5080,1098,181.428571,39.214286,1984
49,Mondale,1984-10-21,25,5128,1198,205.120000,47.920000,1984


## Database Normalization

In [1492]:
# Create dataframe with all data including debate/candidate/performance IDs
candidate_debates_df = pd.merge(debate_dialogue_df, candidates_df, left_on=['actor', 'year'], right_on=['last_name','year'],how='inner')

# Create dataframe of unique candidates to set unique IDs
unique_candidates = pd.DataFrame(candidate_debates_df.sort_values(by='date')['candidate'].unique())
# Reset index for unique candidate IDs
unique_candidates = unique_candidates.reset_index()
# Rename columns
unique_candidates.columns = ['candidate_id', 'name']
# Merge dataframes back together preserving candidate IDs
candidate_debates_df = candidate_debates_df.merge(unique_candidates, left_on='candidate', right_on='name')

candidate_debates_df.head()

,actor,date,speaking_turn_count,total_word_count,unique_word_count,avg_words_per_turn,avg_unique_words_per_turn,year,debate_count,candidate,type,party,last_name,won_election,title,candidate_id,name
0,Nixon,1960-09-26,10,4111,820,411.100000,82.000000,1960,4,Richard Nixon,P,R,Nixon,False,Vice President,0,Richard Nixon
1,Nixon,1960-10-07,12,4336,945,361.333333,78.750000,1960,4,Richard Nixon,P,R,Nixon,False,Vice President,0,Richard Nixon
2,Nixon,1960-10-13,14,4642,934,331.571429,66.714286,1960,4,Richard Nixon,P,R,Nixon,False,Vice President,0,Richard Nixon
3,Nixon,1960-10-21,10,4617,948,461.700000,94.800000,1960,4,Richard Nixon,P,R,Nixon,False,Vice President,0,Richard Nixon
4,Kennedy,1960-09-26,17,4693,953,276.058824,56.058824,1960,4,John F. Kennedy,P,D,Kennedy,True,Senator,1,John F. Kennedy


In [1493]:
# Create dataframe of unique debates to set unique IDs
unique_debates = pd.DataFrame(candidate_debates_df['date'].unique())
# Reset index for unique debates IDs
unique_debates = unique_debates.reset_index()
# Rename columns
unique_debates.columns = ['debate_id', 'date']
# Merge dataframes back together preserving candidate IDs
candidate_debates_df = candidate_debates_df.merge(unique_debates, on='date')

candidate_debates_df.head()

,actor,date,speaking_turn_count,total_word_count,unique_word_count,avg_words_per_turn,avg_unique_words_per_turn,year,debate_count,candidate,type,party,last_name,won_election,title,candidate_id,name,debate_id
0,Nixon,1960-09-26,10,4111,820,411.100000,82.000000,1960,4,Richard Nixon,P,R,Nixon,False,Vice President,0,Richard Nixon,0
1,Kennedy,1960-09-26,17,4693,953,276.058824,56.058824,1960,4,John F. Kennedy,P,D,Kennedy,True,Senator,1,John F. Kennedy,0
2,Nixon,1960-10-07,12,4336,945,361.333333,78.750000,1960,4,Richard Nixon,P,R,Nixon,False,Vice President,0,Richard Nixon,1
3,Kennedy,1960-10-07,13,4403,1003,338.692308,77.153846,1960,4,John F. Kennedy,P,D,Kennedy,True,Senator,1,John F. Kennedy,1
4,Nixon,1960-10-13,14,4642,934,331.571429,66.714286,1960,4,Richard Nixon,P,R,Nixon,False,Vice President,0,Richard Nixon,2


In [1494]:
# Reset index to set as unique performance ID
candidate_debates_df = candidate_debates_df.reset_index()
# Rename column
candidate_debates_df = candidate_debates_df.rename(columns={'index':'performance_id'})
# Drop redundant columns
candidate_debates_df = candidate_debates_df.drop(columns=['year', 'actor', 'last_name', 'name', 'avg_words_per_turn', 'avg_unique_words_per_turn'])

# Preview dataframe
candidate_debates_df.head()

,performance_id,date,speaking_turn_count,total_word_count,unique_word_count,debate_count,candidate,type,party,won_election,title,candidate_id,debate_id
0,0,1960-09-26,10,4111,820,4,Richard Nixon,P,R,False,Vice President,0,0
1,1,1960-09-26,17,4693,953,4,John F. Kennedy,P,D,True,Senator,1,0
2,2,1960-10-07,12,4336,945,4,Richard Nixon,P,R,False,Vice President,0,1
3,3,1960-10-07,13,4403,1003,4,John F. Kennedy,P,D,True,Senator,1,1
4,4,1960-10-13,14,4642,934,4,Richard Nixon,P,R,False,Vice President,0,2


In [1498]:
debates = candidate_debates_df.groupby(['debate_id','date','type'], as_index=False).sum()

debates = debates[['debate_id', 'date', 'type']]

debates.to_csv('./Tables/debates.csv', index=False)

debates.head()

,debate_id,date,type
0,0,1960-09-26,P
1,1,1960-10-07,P
2,2,1960-10-13,P
3,3,1960-10-21,P
4,4,1976-09-23,P


In [1505]:
candidates_grouped = candidate_debates_df.groupby(['candidate_id','candidate','party'], as_index=False).sum()

candidates = candidates_grouped[['candidate_id', 'candidate', 'party']]

candidates = candidates.rename(columns={'candidate':'name'})

candidates.to_csv('./Tables/candidates.csv', index=False)

candidates.head()

,candidate_id,name,party
0,0,Richard Nixon,R
1,1,John F. Kennedy,D
2,2,Gerald Ford,R
3,3,Jimmy Carter,D
4,4,Ronald Reagan,R


In [1510]:
# Combine dataframes to get debate ids/types and dialogue stats
performances = candidate_debates_df[['performance_id', 'debate_id', 'type', 'candidate_id', 'title', 'won_election',
                                     'speaking_turn_count', 'total_word_count', 'unique_word_count']]

# Create calculated column for avg number of words per turn
performances['words_per_turn'] = performances['total_word_count'] / performances['speaking_turn_count']
performances['unique_words_per_turn'] = performances['unique_word_count'] / performances['speaking_turn_count']

# Export performances table as csv
# performances.to_csv('./Tables/performances.csv', index=False)

performances


/Users/glena.dagger/Downloads/anaconda3/envs/PythonDataOne/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/glena.dagger/Downloads/anaconda3/envs/PythonDataOne/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,performance_id,debate_id,type,candidate_id,title,won_election,speaking_turn_count,total_word_count,unique_word_count,words_per_turn,unique_words_per_turn
0,0,0,P,0,Vice President,False,10,4111,820,411.100000,82.000000
1,1,0,P,1,Senator,True,17,4693,953,276.058824,56.058824
2,2,1,P,0,Vice President,False,12,4336,945,361.333333,78.750000
3,3,1,P,1,Senator,True,13,4403,1003,338.692308,77.153846
4,4,2,P,0,Vice President,False,14,4642,934,331.571429,66.714286
...,...,...,...,...,...,...,...,...,...,...,...
87,87,42,P,30,Former Secretary of State,False,82,6662,1410,81.243902,17.195122
88,88,43,VP,32,Senator,False,188,7560,1464,40.212766,7.787234
89,89,43,VP,31,Governor,True,207,6982,1385,33.729469,6.690821
90,90,44,VP,31,Vice President,False,89,6411,1341,72.033708,15.067416


In [ ]:
viewership_table

,Election,Debate,Sponsor,Location,Location.1,Moderators,Viewership,Source
0,1960,First debate,"ABC, CBS, and NBC",WBBM-TV studios,"Chicago, Illinois",Howard K. Smith of CBS,66.4 million,[27]
1,1960,Second debate,"ABC, CBS, and NBC",WRC-TV studios,"Washington, DC",Frank McGee of NBC,61.9 million,[27]
2,1960,Third debate,"ABC, CBS, and NBC",Split-screen telecast with Nixon and panelists...,Split-screen telecast with Nixon and panelists...,Bill Shadel of ABC,63.7 million,[27]
3,1960,Fourth debate,"ABC, CBS, and NBC",ABC Studios,"New York City, New York",Quincy Howe of ABC,60.4 million,[27]
4,1976,First debate,League of Women Voters,Walnut Street Theater,"Philadelphia, Pennsylvania",Edwin Newman of NBC,69.7 million,[28]
5,1976,Second debate,League of Women Voters,Palace of Fine Arts,"San Francisco, California",Pauline Frederick of NPR,63.9 million,[28]
6,1976,Third debate,League of Women Voters,Phi Beta Kappa Memorial Hall at W&M,"Williamsburg, Virginia",Barbara Walters of ABC,62.7 million,[28]
7,1976,VP Debate,League of Women Voters,Alley Theatre,"Houston, Texas",James Hoge of the Chicago Sun-Times,43.2 million,[28]
8,1980,First debate,League of Women Voters,Baltimore Convention Center,"Baltimore, Maryland",Bill Moyers of PBS,NaN,[29]
9,1980,Second debate,League of Women Voters,Public Music Hall,"Cleveland, Ohio",Howard K. Smith of ABC,80.6 million,[29]


In [1511]:
debates

,debate_id,date,type
0,0,1960-09-26,P
1,1,1960-10-07,P
2,2,1960-10-13,P
3,3,1960-10-21,P
4,4,1976-09-23,P
5,5,1976-10-06,P
6,6,1976-10-22,P
7,7,1980-10-28,P
8,8,1980-09-21,P
9,9,1984-10-07,P
